In [129]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def c_generate(x_range):
    bin=len(x_range)
    C=np.random.random((bin,bin))
    for i in range(bin):
        for j in range(bin):
            C[i,j]=abs(x_range[i]-x_range[j])
    return C

def normialise(tem_dist):
    return [tem_dist[i]/sum(tem_dist) for i in range(len(tem_dist))]

def tmp_generator(gamma_dict,num,q_dict,q_num,L):
    bin=gamma_dict[0].shape[0]
    if q_num<=0:
        q=np.matrix(np.ones((bin,bin)))
    else:
        q=q_dict[q_num]
    tmp_gamma=np.zeros((bin,bin))
    tmp_q=np.zeros((bin,bin))
    for i in range(bin):
        for j in range(bin):
            tmp_gamma[i,j]=q.item(i,j)*gamma_dict[num-1].item(i,j)*gamma_dict[num-L-1].item(i,j)/gamma_dict[num-L].item(i,j)
            tmp_q[i,j]=q.item(i,j)*gamma_dict[num-L-1].item(i,j)/gamma_dict[num-L].item(i,j)
    return np.matrix(tmp_gamma),np.matrix(tmp_q)     

def samples_groupby(data):
    # for better complexity
    df=data.groupby(by=['X','S','Y'],as_index=False).sum()
    return df[df['W']!=0]

def assess(bin,f,g,C,V,output):
    output=output.A1.reshape((bin,bin))
    print('sum of violation of f:',sum(abs(np.sum(output,1)-f)))
    print('sum of violation of g:',sum(abs(np.sum(output,0)-g)))
    print('total cost:',sum(sum(output*C)))
    print('entropy:',sum(sum(-output*np.log(output+0.1**3))))
    print('tr violation:',sum(abs(output.T@V)))
    print('============================================')

def plots(x_range,g,f,output):
    fig = plt.figure(figsize=(3,3))
    gs = fig.add_gridspec(2, 2, width_ratios=(4,1), height_ratios=(1,4),left=0.1,right=0.9,bottom=0.1, top=0.9,wspace=0,hspace=0)
    # Create the Axes.
    ax = fig.add_subplot(gs[1, 0])
    ax.pcolormesh(x_range, x_range, output, cmap='Blues')
    ax.set_xlabel(r'supp($X$)',fontsize=10)
    ax.set_ylabel(r'supp($\tilde{X}$)',fontsize=10)#
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax_histx = fig.add_subplot(gs[0, 0], sharex=ax) 
    ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
    #ax_histx.set_title(r'$Pr[x]$',rotation='horizontal')
    #ax_histy.set_title(r'$Pr[\tilde{x}]$')
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histx.tick_params(axis="y", labelleft=False)
    ax_histy.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)
    ax_histx.plot(x_range,g,color='tab:blue')
    ax_histy.plot(f,x_range,color='tab:green') 
    return fig

def v_value(x0,x1,x):
    if x != 0:
        return (x0-x1)/x
    else:
        return 0
    
def newton(fun,dfun,a, stepmax, tol):
    if abs(fun(a))<=tol: return a
    for step in range(1, stepmax+1):
        b=a-fun(a)/dfun(a)
        if abs(fun(b))<=tol:
            return b
        else:
            a = b
    return b 

# simplist
def baseline(C,e,px,ptx,V,K):
    bin=len(px)
    bbm1=np.matrix(np.ones(bin)).T
    #I=np.where(~(V==0))[0].tolist()
    xi=np.exp(-C/e)
    gamma_classic=dict()
    gamma_classic[0]=np.matrix(xi+1.0e-9)
    for repeat in range(K):
        gamma_classic[1+2*repeat]=np.matrix(np.diag((px/(gamma_classic[2*repeat] @ bbm1)).A1))@gamma_classic[2*repeat] #np.diag(dist['x']/sum(gamma_classic.T))@gamma_classic
        gamma_classic[2+2*repeat]=gamma_classic[1+2*repeat]@np.matrix(np.diag((ptx/(gamma_classic[1+2*repeat].T @ bbm1)).A1))

    assess(bin,dist['x'],dist['t_x'],C,V,gamma_classic[2*K])
    return gamma_classic

# our method | total repair
def total_repair(C,e,px,ptx,V,K):
    bin=len(px)
    bbm1=np.matrix(np.ones(bin)).T
    I=np.where(~(V==0))[0].tolist()
    xi=np.exp(-C/e)
    gamma_dict=dict()
    gamma_dict[0]=np.matrix(xi+1.0e-9)
    gamma_dict[1]=np.matrix(np.diag((px/(gamma_dict[0] @ bbm1)).A1))@gamma_dict[0]
    gamma_dict[2]=gamma_dict[1]@np.matrix(np.diag((ptx/(gamma_dict[1].T @ bbm1)).A1))
    # step 3
    J=np.where(~((gamma_dict[2].T @ V).A1 ==0))[0].tolist()
    nu=np.zeros(bin)
    gamma_dict[3]=np.copy(gamma_dict[2])
    for j in J:
        fun = lambda z: sum(gamma_dict[2].item(i,j)*V.item(i)*np.exp(z*V.item(i)) for i in I)
        dfun = lambda z: sum(gamma_dict[2].item(i,j)*(V.item(i))**2*np.exp(z*V.item(i)) for i in I)
        nu = newton(fun,dfun,0.5,stepmax = 25,tol = 1.0e-3) #bisection(fun, -50,50, stepmax = 25, tol = 1.0e-3)
        for i in I:
            gamma_dict[3][i,j]=np.exp(nu*V.item(i))*gamma_dict[2].item(i,j)
    gamma_dict[3]=np.matrix(gamma_dict[3])

    #=========================
    L=3
    q_dict=dict()
    for loop in range(1,K):
        tmp,q_dict[(loop-1)*L+1]=tmp_generator(gamma_dict,loop*L+1,q_dict,(loop-2)*L+1,L) #np.matrix(gamma_dict[3].A1*gamma_dict[0].A1/gamma_dict[1].A1)
        gamma_dict[loop*L+1]=np.matrix(np.diag((px/(tmp @ bbm1)).A1))@tmp

        tmp,q_dict[(loop-1)*L+2]=tmp_generator(gamma_dict,loop*L+2,q_dict,(loop-2)*L+2,L)  #np.matrix(gamma_dict[4].A1*gamma_dict[1].A1/gamma_dict[2].A1)
        gamma_dict[loop*L+2]=tmp@np.matrix(np.diag((ptx/(tmp.T @ bbm1)).A1))

        # step 3
        tmp,q_dict[(loop-1)*L+3]=tmp_generator(gamma_dict,loop*L+3,q_dict,(loop-2)*L+3,L)  #np.matrix(gamma_dict[5].A1*gamma_dict[2].A1/gamma_dict[3].A1)
        J=np.where(~((abs(np.matrix(tmp).T @ V).A1)<=1.0e-5))[0].tolist()
        gamma_dict[loop*L+3]=np.copy(tmp)
        for j in J:
            fun = lambda z: sum(tmp.item(i,j)*V.item(i)*np.exp(z*V.item(i)) for i in I)
            dfun = lambda z: sum(tmp.item(i,j)*(V.item(i))**2*np.exp(z*V.item(i)) for i in I)
            nu = newton(fun,dfun,0.5,stepmax = 25,tol = 1.0e-5) 
            for i in I:
                gamma_dict[loop*L+3][i,j]=np.exp(nu*V.item(i))*tmp.item(i,j)
        gamma_dict[loop*L+3]=np.matrix(gamma_dict[loop*L+3])

    assess(bin,dist['x'],dist['t_x'],C,V,gamma_dict[K*L])
    return gamma_dict

# our method | partial repair
def partial_repair(C,e,px,ptx,V,theta_scale,K):
    bin=len(px)
    bbm1=np.matrix(np.ones(bin)).T
    I=np.where(~(V==0))[0].tolist()
    xi=np.exp(-C/e)
    theta=bbm1*theta_scale
    gamma_dict=dict()
    gamma_dict[0]=np.matrix(xi+1.0e-9)
    gamma_dict[1]=np.matrix(np.diag((px/(gamma_dict[0] @ bbm1)).A1))@gamma_dict[0]
    gamma_dict[2]=gamma_dict[1]@np.matrix(np.diag((ptx/(gamma_dict[1].T @ bbm1)).A1))
    # step 3
    Jplus=np.where(~((gamma_dict[2].T @ V).A1 <=theta.A1))[0].tolist()
    Jminus=np.where(~((gamma_dict[2].T @ V).A1>=-theta.A1))[0].tolist()
    gamma_dict[3]=np.copy(gamma_dict[2])
    for j in Jplus:
        fun = lambda z: sum(gamma_dict[2].item(i,j)*V.item(i)*np.exp(-z*V.item(i)) for i in I)-theta.item(j)
        dfun = lambda z: -sum(gamma_dict[2].item(i,j)*(V.item(i))**2*np.exp(-z*V.item(i)) for i in I)
        nu = newton(fun,dfun,0.5,stepmax = 25,tol = 1.0e-3) #bisection(fun, -50,50, stepmax = 25, tol = 1.0e-3)
        for i in I:
            gamma_dict[3][i,j]=np.exp(-nu*V.item(i))*gamma_dict[2].item(i,j)
    for j in Jminus:
        fun = lambda z: sum(gamma_dict[2].item(i,j)*V.item(i)*np.exp(-z*V.item(i)) for i in I)+theta.item(j)
        dfun = lambda z: -sum(gamma_dict[2].item(i,j)*(V.item(i))**2*np.exp(-z*V.item(i)) for i in I)
        nu = newton(fun,dfun,0.5,stepmax = 25,tol = 1.0e-3) #bisection(fun, -50,50, stepmax = 25, tol = 1.0e-3)
        for i in I:
            gamma_dict[3][i,j]=np.exp(-nu*V.item(i))*gamma_dict[2].item(i,j)
    gamma_dict[3]=np.matrix(gamma_dict[3])

    #=========================
    L=3
    q_dict=dict()
    for loop in range(1,K):
        tmp,q_dict[(loop-1)*L+1]=tmp_generator(gamma_dict,loop*L+1,q_dict,(loop-2)*L+1,L) #np.matrix(gamma_dict[3].A1*gamma_dict[0].A1/gamma_dict[1].A1)
        gamma_dict[loop*L+1]=np.matrix(np.diag((px/(tmp @ bbm1)).A1))@tmp

        tmp,q_dict[(loop-1)*L+2]=tmp_generator(gamma_dict,loop*L+2,q_dict,(loop-2)*L+2,L)  #np.matrix(gamma_dict[4].A1*gamma_dict[1].A1/gamma_dict[2].A1)
        gamma_dict[loop*L+2]=tmp@np.matrix(np.diag((ptx/(tmp.T @ bbm1)).A1))

        # step 3
        tmp,q_dict[(loop-1)*L+3]=tmp_generator(gamma_dict,loop*L+3,q_dict,(loop-2)*L+3,L)  #np.matrix(gamma_dict[5].A1*gamma_dict[2].A1/gamma_dict[3].A1)
        Jplus=np.where(~((np.matrix(tmp).T @ V).A1 <=theta.A1))[0].tolist()
        Jminus=np.where(~((np.matrix(tmp).T @ V).A1>=-theta.A1))[0].tolist()
        gamma_dict[loop*L+3]=np.copy(tmp)
        for j in Jplus:
            fun = lambda z: sum(tmp.item(i,j)*V.item(i)*np.exp(-z*V.item(i)) for i in I)-theta.item(j)
            dfun = lambda z: -sum(tmp.item(i,j)*(V.item(i))**2*np.exp(-z*V.item(i)) for i in I)
            nu = newton(fun,dfun,0.5,stepmax = 25,tol = 1.0e-5) 
            for i in I:
                gamma_dict[loop*L+3][i,j]=np.exp(-nu*V.item(i))*tmp.item(i,j)
        for j in Jminus:
            fun = lambda z: sum(tmp.item(i,j)*V.item(i)*np.exp(-z*V.item(i)) for i in I)+theta.item(j)
            dfun = lambda z: -sum(tmp.item(i,j)*(V.item(i))**2*np.exp(-z*V.item(i)) for i in I)
            nu = newton(fun,dfun,0.5,stepmax = 25,tol = 1.0e-5) 
            for i in I:
                gamma_dict[loop*L+3][i,j]=np.exp(-nu*V.item(i))*tmp.item(i,j)
        gamma_dict[loop*L+3]=np.matrix(gamma_dict[loop*L+3])

    assess(bin,dist['x'],dist['t_x'],C,V,gamma_dict[L*K])
    return gamma_dict

def empirical_distribution(sub,x_range):
    bin=len(x_range)
    distrition=np.zeros(bin)
    for i in range(bin):
        subset=sub[sub['X']==x_range[i]] #bin_value=x_range[i] #sub[(sub['X']>=bin_value)&(sub['X']<bin_value+width)]
        if subset.shape[0]>0:
            distrition[i]=sum(subset['W'])
    if sum(distrition)>0:
        return distrition/sum(distrition)
    else:
        return distrition

def projection(df,coupling_matrix,x_range):
    bin=len(x_range)
    coupling=coupling_matrix.A1.reshape((bin,bin))
    df_t=pd.DataFrame(columns=['X','S','W','Y'])
    for i in range(df.shape[0]):
        orig=df.iloc[i]
        loc=np.where(x_range==orig[0])[0][0]
        rows=np.nonzero(coupling[loc,:])[0]
        sub=pd.DataFrame(columns=['X','W'],index=rows)
        sub['X']=x_range[rows]
        sub['W']=coupling[loc,rows]/(sum(coupling[loc,rows]))*orig[2]
        sub['S']=orig[1]
        sub['Y']=orig[3]
        df_t=pd.concat([df_t, samples_groupby(sub)], ignore_index=True)
    return df_t

def plot_rdist(rdist,x_range):
    plt.plot(x_range,rdist['x'],label=r'$Pr[x]$',color='tab:blue')
    plt.plot(x_range,rdist['x_0'],label=r'$Pr[x|s_0]$',alpha=0.3,color='tab:orange')
    plt.plot(x_range,rdist['x_1'],label=r'$Pr[x|s_1]$',alpha=0.3,color='#9f86c0')
    plt.ylabel('PMF',fontsize=14)
    plt.xlabel(r'$supp(X)=supp(\tilde{X})$',fontsize=20)
    plt.legend()
    return plt

# def DisparateImpact(X_test,y_test,y_pred):
#     df_test=pd.DataFrame(np.concatenate((X_test,y_test.reshape(-1,1),y_pred.reshape(-1,1)), axis=1),columns=['X','S','W','Y','f'])
#     numerator=sum(df_test[(df_test['S']==0)&(df_test['f']==1)&(df_test['Y']==1)]['W'])/sum(df_test[(df_test['S']==0)&(df_test['Y']==1)]['W'])
#     denominator=sum(df_test[(df_test['S']==1)&(df_test['f']==1)&(df_test['Y']==1)]['W'])/sum(df_test[(df_test['S']==1)&(df_test['Y']==1)]['W'])
#     return numerator/denominator

# def DisparateImpact(X_test,y_pred):
#     df_test=pd.DataFrame(np.concatenate((X_test,y_pred.reshape(-1,1)), axis=1),columns=['X','S','W','f'])
#     numerator=sum(df_test[(df_test['S']==0)&(df_test['f']==1)]['W'])/sum(df_test[df_test['S']==0]['W'])
#     denominator=sum(df_test[(df_test['S']==1)&(df_test['f']==1)]['W'])/sum(df_test[df_test['S']==1]['W'])
#     if numerator==denominator:
#         return 1
#     return numerator/denominator

def DisparateImpact(X_test,y_pred):
    dim=X_test.shape[1]-2
    df_test=pd.DataFrame(np.concatenate((X_test,y_pred.reshape(-1,1)), axis=1),columns=[*range(dim)]+['S','W','f'])
    numerator=sum(df_test[(df_test['S']==0)&(df_test['f']==1)]['W'])/sum(df_test[df_test['S']==0]['W'])
    denominator=sum(df_test[(df_test['S']==1)&(df_test['f']==1)]['W'])/sum(df_test[df_test['S']==1]['W'])
    if numerator==denominator:
        return 1
    return numerator/denominator

def rdata_analysis(rdata,x_range,x_name):
    rdist=dict()
    pivot=pd.pivot_table(rdata,index=x_name,values=['W'],aggfunc=[np.sum])[('sum','W')]
    pivot0=pd.pivot_table(rdata[rdata['S']==0],index=x_name,values=['W'],aggfunc=[np.sum])[('sum','W')]
    pivot1=pd.pivot_table(rdata[rdata['S']==1],index=x_name,values=['W'],aggfunc=[np.sum])[('sum','W')]
    rdist['x']= np.array([pivot[i] for i in x_range])/sum([pivot[i] for i in x_range]) #empirical_distribution(rdata,x_range)
    rdist['x_0']=np.array([pivot0[i] if i in list(pivot0.index) else 0 for i in x_range])/sum([pivot0[i] if i in list(pivot0.index) else 0 for i in x_range]) #empirical_distribution(rdata[rdata['S']==0],x_range)
    rdist['x_1']=np.array([pivot1[i] if i in list(pivot1.index) else 0 for i in x_range])/sum([pivot1[i] if i in list(pivot1.index) else 0 for i in x_range]) #empirical_distribution(rdata[rdata['S']==1],x_range)
    return rdist

def c_generate_higher(x_range,weight):
    bin=len(x_range)
    dim=len(x_range[0])
    C=np.random.random((bin,bin))
    for i in range(bin):
        for j in range(bin):
            C[i,j]=sum(weight[d]*abs(x_range[i][d]-x_range[j][d]) for d in range(dim))
    return C

def projection_higher(df,coupling_matrix,x_range,x_list,var_list):
    df=df.drop(columns=x_list)
    dim=len(x_list)
    bin=len(x_range)
    arg_list=[elem for elem in var_list if elem not in x_list]
    df=df[arg_list+['X','S','W','Y']]
    coupling=coupling_matrix.A1.reshape((bin,bin))
    df_t=pd.DataFrame(columns=arg_list+['X','S','W','Y'])
    for i in range(df.shape[0]):
        orig=df.iloc[i]
        loc=np.where([x_range[i]==orig['X'] for i in range(bin)])[0][0]
        rows=np.nonzero(coupling[loc,:])[0]
        sub_dict={'X':[x_range[r] for r in rows],'W':list(coupling[loc,rows]/(sum(coupling[loc,rows]))*orig['W'])}
        sub_dict.update({var:[orig[var]]*len(rows) for var in arg_list+['S','Y']})
        sub=pd.DataFrame(data=sub_dict, index=rows)
        df_t=pd.concat([df_t,sub],ignore_index=True)#pd.concat([df_t,samples_groupby(sub,x_list)], ignore_index=True)
    df_t=df_t.groupby(by=list(chain(*[arg_list,'X','S','Y'])),as_index=False).sum()
    for d in range(dim):
        df_t[x_list[d]]=[df_t['X'][r][d] for r in range(df_t.shape[0])]
    return df_t[var_list+['S','W','Y']]

In [252]:
var_list=['age','education-num','capitalgain','capitalloss','hoursperweek','class']
weight=[4,1,4,4,4]
var_dim=len(var_list)
pa='sex'
pa_dict={'Male':1,'Female':0}

messydata=pd.read_csv('C:/Users/zhouq/Documents/optimal_transport/adult_csv.csv',usecols=var_list+[pa])
messydata[pa]=messydata[pa].replace(pa_dict)
messydata['class']=messydata['class'].replace({'>50K':1,'<=50K':0})
messydata=messydata[(messydata[pa]==1)|(messydata[pa]==0)]
for col in var_list+[pa]:
    messydata[col]=messydata[col].astype('category')
messydata['W']=1

X=messydata[var_list[:-1]+[pa,'W']].to_numpy() # [X,S,W]
y=messydata['class'].to_numpy() #[Y]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [253]:
clf=RandomForestClassifier(max_depth=5, random_state=0).fit(X_train[:,0:var_dim-1],y_train)
y_pred=clf.predict(X_test[:,0:var_dim-1])
new_row = pd.Series({'DI':DisparateImpact(X_test,y_pred),
                    'f1 macro':f1_score(y_test, y_pred, average='macro'),
                    'f1 micro':f1_score(y_test, y_pred, average='micro'),
                    'f1 weighted':f1_score(y_test, y_pred, average='weighted'),
                    'method':'origin','test set':'origin'})
print(new_row)

DI             0.433099
f1 macro        0.68512
f1 micro       0.818535
f1 weighted    0.791945
method           origin
test set         origin
dtype: object


In [254]:
df_test=pd.DataFrame(np.concatenate((X_test,y_test.reshape(-1,1),y_pred.reshape(-1,1)), axis=1),columns=var_list[:-1]+[pa,'W','class','f'])

In [255]:
x_range=list(pd.pivot_table(df_test,index=var_list[:-1],values=['W']).index) # class in the end

In [256]:
df_test['X']=[tuple(df_test[var_list[:-1]].values[r]) for r in range(df_test.shape[0])]
df_test=df_test.rename(columns={pa:'S','class':'Y'})
dist=rdata_analysis(df_test[['X','S','W']],x_range,'X')

bin=len(x_range)
if dim>1:
    C=c_generate_higher(x_range,weight)
else:
    C=c_generate(x_range)
dist['t_x']=dist['x_0']*0.5+dist['x_1']*0.5 # #dist['x'] #
dist['v']=[(dist['x_0'][i]-dist['x_1'][i])/dist['x'][i] for i in range(bin)]
px=np.matrix(dist['x']).T
ptx=np.matrix(dist['t_x']).T
V=np.matrix(dist['v']).T

In [ ]:
def projection_higher(df,coupling_matrix,x_range,x_list,var_list):
    df=df.drop(columns=x_list)
    dim=len(x_list)
    bin=len(x_range)
    arg_list=[elem for elem in var_list if elem not in x_list]
    df=df[arg_list+['X','S','W','Y']]
    coupling=coupling_matrix.A1.reshape((bin,bin))
    df_t=pd.DataFrame(columns=arg_list+['X','S','W','Y'])
    for i in range(df.shape[0]):
        orig=df.iloc[i]
        loc=np.where([x_range[i]==orig['X'] for i in range(bin)])[0][0]
        rows=np.nonzero(coupling[loc,:])[0]
        sub_dict={'X':[x_range[r] for r in rows],'W':list(coupling[loc,rows]/(sum(coupling[loc,rows]))*orig['W'])}
        sub_dict.update({var:[orig[var]]*len(rows) for var in arg_list+['S','Y']})
        sub=pd.DataFrame(data=sub_dict, index=rows)
        df_t=pd.concat([df_t,sub],ignore_index=True)#pd.concat([df_t,samples_groupby(sub,x_list)], ignore_index=True)
    df_t=df_t.groupby(by=list(chain(*[arg_list,'X','S','Y'])),as_index=False).sum()
    for d in range(dim):
        df_t[x_list[d]]=[df_t['X'][r][d] for r in range(df_t.shape[0])]
    return df_t[var_list+['S','W','Y']]

In [217]:
def postprocess(df_test,coupling_matrix,x_range,x_range_pred):
    bin=len(x_range)
    coupling=coupling_matrix.A1.reshape((bin,bin))
    x_pred_repaired=[]
    for loc in range(bin):
        #loc=np.where([x_range[i]==orig['X'] for i in range(bin)])[0][0]
        rows=np.nonzero(coupling[loc,:])[0]
        p0=sum(x_range_pred[:,0]*coupling[loc,rows]/(sum(coupling[loc,rows])))
        p1=sum(x_range_pred[:,1]*coupling[loc,rows]/(sum(coupling[loc,rows])))
        if p0>p1:
            x_pred_repaired+=[0]
        else:
            x_pred_repaired+=[1]
    pred_dict=dict(zip(x_range,x_pred_repaired))
    f_repaired=[pred_dict[df_test['X'][i]] for i in range(df_test.shape[0])]
    return f_repaired

In [257]:
pred_dict=dict(zip(x_range,x_pred_repaired))
print(pred_dict)

{(0, 1, 0, 0, 1): 0, (0, 1, 0, 0, 2): 0, (0, 1, 0, 0, 3): 0, (0, 2, 0, 0, 0): 0, (0, 2, 0, 0, 2): 0, (0, 2, 0, 0, 3): 0, (0, 3, 0, 0, 0): 0, (0, 3, 0, 0, 1): 0, (0, 3, 0, 0, 2): 0, (0, 3, 0, 0, 3): 0, (0, 3, 1, 0, 2): 0, (0, 4, 0, 0, 0): 0, (0, 4, 0, 0, 1): 1, (0, 4, 0, 0, 2): 0, (0, 4, 0, 0, 3): 0, (0, 4, 0, 0, 4): 0, (0, 4, 1, 0, 2): 0, (0, 5, 0, 0, 0): 0, (0, 5, 0, 0, 1): 0, (0, 5, 0, 0, 2): 0, (0, 5, 0, 0, 3): 0, (0, 5, 0, 0, 4): 0, (0, 5, 1, 0, 2): 0, (0, 6, 0, 0, 0): 0, (0, 6, 0, 0, 1): 0, (0, 6, 0, 0, 2): 0, (0, 6, 0, 0, 3): 0, (0, 6, 0, 0, 4): 0, (0, 6, 0, 2, 0): 0, (0, 6, 0, 2, 2): 0, (0, 6, 1, 0, 0): 0, (0, 6, 1, 0, 1): 0, (0, 7, 0, 0, 0): 0, (0, 7, 0, 0, 1): 0, (0, 7, 0, 0, 2): 0, (0, 7, 0, 0, 3): 0, (0, 7, 0, 0, 4): 0, (0, 7, 0, 1, 2): 0, (0, 7, 0, 2, 0): 0, (0, 7, 0, 2, 2): 0, (0, 7, 1, 0, 0): 0, (0, 7, 1, 0, 1): 0, (0, 7, 1, 0, 2): 0, (0, 7, 3, 0, 3): 0, (0, 8, 0, 0, 0): 0, (0, 8, 0, 0, 1): 0, (0, 8, 0, 0, 2): 0, (0, 8, 0, 0, 3): 0, (0, 8, 0, 2, 1): 0, (0, 8, 0, 2, 2): 0,

In [240]:
bin=len(x_range)
coupling=coupling_matrix.A1.reshape((bin,bin))
x_pred_repaired=[]
for loc in range(bin):
    #loc=np.where([x_range[i]==orig['X'] for i in range(bin)])[0][0]
    rows=np.nonzero(coupling[loc,:])[0]
    p0=sum(x_range_pred[:,0]*coupling[loc,rows]/(sum(coupling[loc,rows])))
    p1=sum(x_range_pred[:,1]*coupling[loc,rows]/(sum(coupling[loc,rows])))
    if p0>p1:
        x_pred_repaired+=[0]
    else:
        x_pred_repaired+=[1]
pred_dict=dict(zip(x_range,x_pred_repaired))
f_repaired=[pred_dict[df_test['X'][i]] for i in range(df_test.shape[0])]

In [267]:
bin

1054

In [258]:
x_range_pred=clf.predict_proba(np.array(x_range))
f_repaired_base=postprocess(df_test,baseline(C,e,px,ptx,V,K)[K*2],x_range,x_range_pred)

sum of violation of f: 0.004926109435082405
sum of violation of g: 8.32009970901798e-15
total cost: 1.0683239495301629
entropy: 4.9094473276800565
tr violation: [[0.51980833]]


In [262]:
f_repaired_part=postprocess(df_test,partial_repair(C,e,px,ptx,V,1.0e-2,K)[K*3],x_range,x_range_pred)

sum of violation of f: 0.004900544494769922
sum of violation of g: 2.4629600878234902e-06
total cost: 1.2275818620579702
entropy: 4.935584707882258
tr violation: [[0.50588706]]


In [ ]:
df_test.insert(0,"f_base",f_repaired_base)
df_test.insert(0,"f_part",f_repaired_part)

In [263]:
# X_test=df_test[var_list[:-1]+['S','W']]
# y_test=df_test['Y']
# y_pred=df_test['f']
y_pred_base=np.array(f_repaired_base)
y_pred_part=np.array(f_repaired_part)
new_row=pd.Series({'DI':DisparateImpact(X_test,y_pred),
                    'f1 macro':f1_score(y_test, y_pred, average='macro'),
                    'f1 micro':f1_score(y_test, y_pred, average='micro'),
                    'f1 weighted':f1_score(y_test, y_pred, average='weighted'),
                    'method':'origin'})
new_row_base = pd.Series({'DI':DisparateImpact(X_test,y_pred_base),
                'f1 macro':f1_score(y_test, y_pred_base, average='macro'),
                'f1 micro':f1_score(y_test, y_pred_base, average='micro'),
                'f1 weighted':f1_score(y_test, y_pred_base, average='weighted'),
                'method':'baseline'})
new_row_part = pd.Series({'DI':DisparateImpact(X_test,y_pred_part),
                'f1 macro':f1_score(y_test, y_pred_part, average='macro'),
                'f1 micro':f1_score(y_test, y_pred_part, average='micro'),
                'f1 weighted':f1_score(y_test, y_pred_part, average='weighted'),
                'method':'partial repair'})

In [264]:
print(new_row)

DI             0.433099
f1 macro        0.68512
f1 micro       0.818535
f1 weighted    0.791945
method           origin
dtype: object


In [265]:
print(new_row_base)

DI              0.49089
f1 macro       0.654493
f1 micro       0.813485
f1 weighted    0.776648
method         baseline
dtype: object


In [266]:
print(new_row_part)

DI                   0.503221
f1 macro              0.66277
f1 micro             0.814168
f1 weighted          0.780536
method         partial repair
dtype: object
